In [1]:
from typing import Dict, Optional, Any, List, Self, Literal, ClassVar, Union
from uuid import UUID, uuid4
from pydantic import BaseModel, Field, model_validator, computed_field, field_validator
from enum import Enum
from dnd.values import ModifiableValue, AdvantageStatus, CriticalStatus, AutoHitStatus, StaticValue,NumericalModifier, ContextualValue

from dnd.blocks import (
    BaseBlock, AbilityScores, SavingThrowSet, Health, 
    Equipped, Speed, ActionEconomy,SkillSet,SkillName
)
from dnd.values import ModifiableValue
from dnd.modifiers import (
    NumericalModifier, DamageType, ResistanceStatus, 
    ContextAwareCondition, BaseObject
)

from dnd.equipment import (
    Armor, Weapon, Shield, BodyArmor, Gauntlets, Greaves,
    Boots, Amulet, Ring, Cloak, Helmet, BodyPart
)
from dnd.dice import Dice, RollType, DiceRoll

In [2]:
class Entity(BaseBlock):
    """ Base class for dnd entities in the game it acts as container for blocks and implements common functionalities that
    require interactions between blocks """
    
    name: str = Field(default="Entity")
    ability_scores: AbilityScores = Field(default_factory=lambda: AbilityScores.create(source_entity_uuid=uuid4()))
    skill_set: SkillSet = Field(default_factory=lambda: SkillSet.create(source_entity_uuid=uuid4()))
    saving_throws: SavingThrowSet = Field(default_factory=lambda: SavingThrowSet.create(source_entity_uuid=uuid4()))
    health: Health = Field(default_factory=lambda: Health.create(source_entity_uuid=uuid4()))
    equipped: Equipped = Field(default_factory=lambda: Equipped.create(source_entity_uuid=uuid4()))
    speed: Speed = Field(default_factory=lambda: Speed.create(source_entity_uuid=uuid4()))
    action_economy: ActionEconomy = Field(default_factory=lambda: ActionEconomy.create(source_entity_uuid=uuid4()))
    proficiency_bonus: ModifiableValue = Field(default_factory=lambda: ModifiableValue.create(source_entity_uuid=uuid4(),value_name="proficiency_bonus",base_value=2))
    @classmethod
    def create(cls, source_entity_uuid: UUID, name: str = "Entity") -> 'Entity':
        """
        Create a new Entity instance with the given parameters. All sub-blocks will share
        the same source_entity_uuid as the entity itself.

        Args:
            source_entity_uuid (UUID): The UUID that will be used as both the entity's UUID and source_entity_uuid
            name (str): The name of the entity. Defaults to "Entity"

        Returns:
            Entity: The newly created Entity instance
        """
        return cls(
            uuid=source_entity_uuid,
            source_entity_uuid=source_entity_uuid,
            name=name)
            
    



In [3]:
class AttackOutcome(str, Enum):
    HIT = "Hit"
    MISS = "Miss"
    CRIT = "Crit"

class RollType(str, Enum):
    DAMAGE = "Damage"
    ATTACK = "Attack"
    SAVE = "Save"
    SKILL = "Skill"

class DiceRoll(BaseModel):
    """
    Represents the result of a dice roll.

    This class stores information about a specific dice roll, including the roll type,
    results, and various statuses that may affect the roll.

    Attributes:
        roll_uuid (UUID): Unique identifier for this roll. Automatically generated if not provided.
        dice_uuid (UUID): Unique identifier of the Dice object that produced this roll.
        roll_type (RollType): The type of roll (e.g., DAMAGE, ATTACK, SAVE, SKILL).
        results (Union[List[int], int]): The individual die results or a single result.
        total (int): The total value of the roll, including any bonuses.
        bonus (int): Any additional bonus applied to the roll.
        advantage_status (AdvantageStatus): The advantage status of the roll.
        critical_status (CriticalStatus): The critical status of the roll.
        auto_hit_status (AutoHitStatus): The auto-hit status of the roll.
        source_entity_uuid (UUID): UUID of the entity that made the roll.
        target_entity_uuid (Optional[UUID]): UUID of the target entity, if applicable.
        attack_outcome (Optional[AttackOutcome]): The outcome of an attack roll, if applicable.

    Class Attributes:
        _registry (ClassVar[Dict[UUID, 'DiceRoll']]): A class-level registry to store all instances.

    Methods:
        get(cls, uuid: UUID) -> Optional['DiceRoll']:
            Retrieve a DiceRoll instance from the registry by its UUID.
        unregister(cls, uuid: UUID) -> None:
            Remove a DiceRoll instance from the class registry.
    """

    _registry: ClassVar[Dict[UUID, 'DiceRoll']] = {}

    roll_uuid: UUID = Field(
        default_factory=uuid4,
        description="Unique identifier for this roll. Automatically generated if not provided."
    )
    dice_uuid: UUID = Field(
        ...,
        description="Unique identifier of the Dice object that produced this roll."
    )
    roll_type: RollType = Field(
        ...,
        description="The type of roll (e.g., DAMAGE, ATTACK, SAVE, SKILL)."
    )
    results: Union[List[int], int] = Field(
        ...,
        description="The individual die results or a single result."
    )
    total: int = Field(
        ...,
        description="The total value of the roll, including any bonuses."
    )
    bonus: int = Field(
        ...,
        description="Any additional bonus applied to the roll."
    )
    advantage_status: AdvantageStatus = Field(
        ...,
        description="The advantage status of the roll."
    )
    critical_status: CriticalStatus = Field(
        ...,
        description="The critical status of the roll."
    )
    auto_hit_status: AutoHitStatus = Field(
        ...,
        description="The auto-hit status of the roll."
    )
    source_entity_uuid: UUID = Field(
        ...,
        description="UUID of the entity that made the roll."
    )
    target_entity_uuid: Optional[UUID] = Field(
        default=None,
        description="UUID of the target entity, if applicable."
    )
    attack_outcome: Optional[AttackOutcome] = Field(
        default=None,
        description="The outcome of an attack roll, if applicable."
    )

    def __init__(self, **data):
        super().__init__(**data)
        self.__class__._registry[self.roll_uuid] = self

    @classmethod
    def get(cls, uuid: UUID) -> Optional['DiceRoll']:
        """
        Retrieve a DiceRoll instance from the registry by its UUID.

        Args:
            uuid (UUID): The UUID of the DiceRoll to retrieve.

        Returns:
            Optional[DiceRoll]: The DiceRoll instance if found, None otherwise.
        """
        return cls._registry.get(uuid)

In [4]:
goblin = Entity.create(source_entity_uuid=uuid4(),name="Goblin")
skeleton = Entity.create(source_entity_uuid=uuid4(),name="Skeleton")
print(goblin.source_entity_uuid)
print(goblin.uuid)
print(skeleton.source_entity_uuid)
print(skeleton.uuid)
goblin.set_target_entity(goblin.uuid)


e0af93fc-90d5-40d5-b644-4f70b8369988
e0af93fc-90d5-40d5-b644-4f70b8369988
e0312caa-5b5f-4fc7-b403-f220bacae207
e0312caa-5b5f-4fc7-b403-f220bacae207


In [5]:
#try to compute the modifieable value for a skill roll
# gotta get a) the proficiency bonus b) the ability modifier c) the skill bonus
# we want to keep them as modifiable value and combine them together
assert isinstance(goblin, Entity)
proficiency_modifier = goblin.proficiency_bonus
extracted_skill = goblin.skill_set.get_skill("acrobatics")
skill_modifier = extracted_skill.skill_bonus
extracted_ability = goblin.ability_scores.get_ability(extracted_skill.ability)
ability_modifier = extracted_ability.ability_score
#oprint all source entity uuids
print(f"Proficiency Bonus: {proficiency_modifier.source_entity_uuid}")
print(f"Skill Bonus: {skill_modifier.source_entity_uuid}")
print(f"Ability Modifier: {ability_modifier.source_entity_uuid}")


Proficiency Bonus: e0af93fc-90d5-40d5-b644-4f70b8369988
Skill Bonus: e0af93fc-90d5-40d5-b644-4f70b8369988
Ability Modifier: e0af93fc-90d5-40d5-b644-4f70b8369988


In [6]:
proficiency_modifier.self_static.target_entity_uuid


UUID('e0af93fc-90d5-40d5-b644-4f70b8369988')

In [7]:
skill_modifier.self_static.target_entity_uuid

UUID('e0af93fc-90d5-40d5-b644-4f70b8369988')

In [8]:
def skill_advantage_status(proficiency_bonus: ModifiableValue, skill_bonus: ModifiableValue, ability_modifier: ModifiableValue) -> AdvantageStatus:
    merged_value = proficiency_bonus.combine_values([skill_bonus])
    return merged_value.advantage
advantage_status = skill_advantage_status(proficiency_modifier, skill_modifier, ability_modifier)
print(advantage_status)

Validating others source id e0af93fc-90d5-40d5-b644-4f70b8369988 against self source e0af93fc-90d5-40d5-b644-4f70b8369988 for modifer  with name proficiency_bonus and uuid 3c176a8b-da27-475c-9d9e-571fd002c051 and type <class 'dnd.values.ModifiableValue'> against <class 'dnd.values.ModifiableValue'> with name Skill Bonus and uuid 43543e8b-66de-4224-92a8-4dbcc082f350
AdvantageStatus.NONE


In [9]:
#formatetd print of the bonus
print(f"Proficiency Bonus: {proficiency_modifier.source_entity_uuid}")
print(f"Skill Bonus: {skill_modifier.source_entity_uuid}")
print(f"Ability Modifier: {ability_modifier.source_entity_uuid}")



Proficiency Bonus: e0af93fc-90d5-40d5-b644-4f70b8369988
Skill Bonus: e0af93fc-90d5-40d5-b644-4f70b8369988
Ability Modifier: e0af93fc-90d5-40d5-b644-4f70b8369988


In [10]:
proficiency_modifier.normalized_score

2

In [11]:
from typing import Tuple, Optional
import random

def roll_d20(advantage_status: AdvantageStatus) -> Tuple[int,Optional[int]]:
        num_rolls = 1 if advantage_status == AdvantageStatus.NONE else 2
        rolls = [random.randint(1,20) for _ in range(num_rolls)]
        if advantage_status == AdvantageStatus.ADVANTAGE:
            return max(rolls), min(rolls)
        elif advantage_status == AdvantageStatus.DISADVANTAGE:
            return min(rolls), max(rolls)
        else:
            return rolls[0], None
        
        
def skill_roll_bonus(skill_name: SkillName, entity: Entity) -> DiceRoll:
    extracted_skill = entity.skill_set.get_skill(skill_name)
    skill_modifier = extracted_skill.skill_bonus
    extracted_ability = entity.ability_scores.get_ability(extracted_skill.ability)
    ability_modifier = extracted_ability.ability_score



        

       
    proficiency_modifier = entity.proficiency_bonus

    

    return DiceRoll(
        roll_type=RollType.SKILL,
        results=,
        source_entity_uuid=entity.uuid,
        target_entity_uuid=None,
        attack_outcome=None
    )



SyntaxError: expected argument value expression (2776065904.py, line 32)